In [1]:
import numpy as np

In [2]:
def get_model(save_name='ResNet50.hdh5', image_shape=(224, 224, 3)):
    import os
    new_model = not os.path.exists(save_name)
    if new_model:
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(
            include_top=False, weights='imagenet',
            input_tensor=None, input_shape=image_shape, pooling='max'
        )
        model.save(save_name)
    else:
        from keras.models import load_model
        model = load_model(save_name)
    return model, new_model

In [3]:
initial_model, new_model = get_model()
if new_model:
    print('new model')
else:
    print('old model')

Using TensorFlow backend.


new model


In [4]:
def get_data(image_shape=(224, 224, 3)):
    from skimage.transform import resize
    from skimage.io import imread
    import os
    images = []
    labels = []
    with open('public_data/00_input/train/gt.csv') as navigate_file:
        for i, line in enumerate(navigate_file):
            if i == 0:
                continue
            image_name, label = line.split(',')                
            images.append(resize(
                imread(os.path.join(
                    'public_data/00_input/train/images/',
                    image_name
                )),
                image_shape,
                mode='reflect'
            ))
            labels.append(int(label))
    return np.array(images), np.array(labels)

In [5]:
x, y = get_data()

In [6]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [7]:
num_classes = len(set(y))
y = np.eye(num_classes)[y]
train_x, test_x, train_y, test_y = train_test_split(
    x, y, test_size=0.3, random_state=42
)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

In [9]:
regularization_lambda = 0.0005

last = initial_model.output

prediction = Dense(
    num_classes, activation='softmax',
    kernel_regularizer=regularizers.l2(regularization_lambda),
    bias_regularizer=regularizers.l2(regularization_lambda)
)(last)

model = Model(initial_model.input, prediction)

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=0.00005),
    metrics=['categorical_accuracy']
)

In [11]:
batch_size = 16
model.fit_generator(
    data_generator.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=int(len(train_y) / batch_size), epochs=300,
    validation_data=(test_x, test_y)
)

Epoch 1/300
109/109 [==============================] - 83s - loss: 3.1742 - categorical_accuracy: 0.2339 - val_loss: 4.3589 - val_categorical_accuracy: 0.0200
Epoch 2/300
109/109 [==============================] - 74s - loss: 1.3386 - categorical_accuracy: 0.6865 - val_loss: 4.1410 - val_categorical_accuracy: 0.0213
Epoch 3/300
109/109 [==============================] - 74s - loss: 0.7569 - categorical_accuracy: 0.8271 - val_loss: 4.0650 - val_categorical_accuracy: 0.0253
Epoch 4/300
109/109 [==============================] - 75s - loss: 0.5044 - categorical_accuracy: 0.8983 - val_loss: 3.9498 - val_categorical_accuracy: 0.0467
Epoch 5/300
109/109 [==============================] - 75s - loss: 0.3404 - categorical_accuracy: 0.9390 - val_loss: 3.6996 - val_categorical_accuracy: 0.1093
Epoch 6/300
109/109 [==============================] - 75s - loss: 0.2427 - categorical_accuracy: 0.9648 - val_loss: 3.3444 - val_categorical_accuracy: 0.1840
Epoch 7/300
109/109 [=========================

KeyboardInterrupt: 

In [12]:
model.save('0.71_score_model.hdh5')